#Recreate paper Twitter Sentiment Analysis

---

Source of paper: https://paperswithcode.com/paper/twitter-sentiment-analysis-1

Source of code: https://github.com/Vedurumudi-Priyanka/Twitter-Sentiment-Analysis

Source of dataset: https://github.com/shuttlesworthNEO/twitter-naive-sentiwordnet

Source of glove.twitter.27b.25d.txt: https://drive.google.com/drive/folders/1LRRlFq2pgk5APevQjfV71_8LB63-YMG8?usp=sharing

Library

In [10]:
!pip2 install numpy
!pip2 install scikit-learn
!pip2 install scipy
!pip2 install nltk
!pip2 install tensorflow
!pip2 install xgboost

utils.py

In [11]:
import sys
import pickle
import random


def file_to_wordset(filename):
    ''' Converts a file with a word per line to a Python set '''
    words = []
    with open(filename, 'r') as f:
        for line in f:
            words.append(line.strip())
    return set(words)


def write_status(i, total):
    ''' Writes status of a process to console '''
    sys.stdout.write('\r')
    sys.stdout.write('Processing %d/%d' % (i, total))
    sys.stdout.flush()


def save_results_to_csv(results, csv_file):
    ''' Save list of type [(tweet_id, positive)] to csv in Kaggle format '''
    with open(csv_file, 'w') as csv:
        csv.write('id,prediction\n')
        for tweet_id, pred in results:
            csv.write(tweet_id)
            csv.write(',')
            csv.write(str(pred))
            csv.write('\n')


def top_n_words(pkl_file_name, N, shift=0):
    """
    Returns a dictionary of form {word:rank} of top N words from a pickle
    file which has a nltk FreqDist object generated by stats.py
    Args:
        pkl_file_name (str): Name of pickle file
        N (int): The number of words to get
        shift: amount to shift the rank from 0.
    Returns:
        dict: Of form {word:rank}
    """
    with open(pkl_file_name, 'rb') as pkl_file:
        freq_dist = pickle.load(pkl_file)
    most_common = freq_dist.most_common(N)
    words = {p[0]: i + shift for i, p in enumerate(most_common)}
    return words


def top_n_bigrams(pkl_file_name, N, shift=0):
    """
    Returns a dictionary of form {bigram:rank} of top N bigrams from a pickle
    file which has a Counter object generated by stats.py
    Args:
        pkl_file_name (str): Name of pickle file
        N (int): The number of bigrams to get
        shift: amount to shift the rank from 0.
    Returns:
        dict: Of form {bigram:rank}
    """
    with open(pkl_file_name, 'rb') as pkl_file:
        freq_dist = pickle.load(pkl_file)
    most_common = freq_dist.most_common(N)
    bigrams = {p[0]: i for i, p in enumerate(most_common)}
    return bigrams


def split_data(tweets, validation_split=0.1):
    """Split the data into training and validation sets
    Args:
        tweets (list): list of tuples
        validation_split (float, optional): validation split %
    Returns:
        (list, list): training-set, validation-set
    """
    index = int((1 - validation_split) * len(tweets))
    random.shuffle(tweets)
    return tweets[:index], tweets[index:]

Preprocess Run preprocess.py Train

In [12]:
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/preprocess.py /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train.csv

Processing 100000/100000
Saved processed tweets to: /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv


Preprocess Run preprocess.py Test

In [13]:
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/preprocess.py /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test.csv

Processing 300000/300000
Saved processed tweets to: /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv


Preprocess Run stats.py Train

In [14]:
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/stats.py /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv

Processing 100000/100000
Calculating frequency distribution
Saved uni-frequency distribution to /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl
Saved bi-frequency distribution to /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl

[Analysis Statistics]
Tweets => Total: 100000, Positive: 56462, Negative: 43538
User Mentions => Total: 88955, Avg: 0.8895, Max: 12
URLs => Total: 3599, Avg: 0.0360, Max: 4
Emojis => Total: 1184, Positive: 997, Negative: 187, Avg: 0.0118, Max: 5
Words => Total: 1192617, Unique: 50377, Avg: 11.9262, Max: 40, Min: 0
Bigrams => Total: 1093149, Unique: 385105, Avg: 10.9315


Baseline

In [15]:
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/baseline.py TRAIN = True

Correct = 65.35%


Naive Bayes

In [16]:
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/naivebayes.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7741/10000 = 77.4100 %


Maximum Entropy

In [17]:
import nltk
import random
import pickle
import sys
import numpy as np

TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
USE_BIGRAMS = False
TRAIN = True


def get_data_from_file(file_name, isTrain=True):
    data = []
    with open(train_csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if isTrain:
                tag = line.split(',')[1]
                bag_of_words = line.split(',')[2].split()
                if USE_BIGRAMS:
                    bag_of_words_bigram = list(nltk.bigrams(line.split(',')[2].split()))
                    bag_of_words = bag_of_words+bag_of_words_bigram
            else :
                tag = '5'
                bag_of_words = line.split(',')[1].split()
                if USE_BIGRAMS:
                    bag_of_words_bigram = list(nltk.bigrams(line.split(',')[1].split()))
                    bag_of_words = bag_of_words+bag_of_words_bigram
            data.append((bag_of_words, tag))
    return data

def split_data(tweets, validation_split=0.1):
    index = int((1 - validation_split) * len(tweets))
    random.shuffle(tweets)
    return tweets[:index], tweets[index:]

def list_to_dict(words_list):
    return dict([(word, True) for word in words_list])

if __name__ == '__main__':
    train = True
    np.random.seed(1337)
    train_csv_file = TRAIN_PROCESSED_FILE
    test_csv_file = TEST_PROCESSED_FILE
    train_data = get_data_from_file(train_csv_file, isTrain=True)
    train_set, validation_set = split_data(train_data)
    training_set_formatted = [(list_to_dict(element[0]), element[1]) for element in train_set]
    validation_set_formatted = [(list_to_dict(element[0]), element[1]) for element in validation_set]
    numIterations = 1
    algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[1]
    classifier = nltk.MaxentClassifier.train(training_set_formatted, algorithm, max_iter=numIterations)
    classifier.show_most_informative_features(10)
    count = int(0)
    for review in validation_set_formatted:
        label = review[1]
        text = review[0]
        determined_label = classifier.classify(text)
        #print(determined_label, label)
        if determined_label!=label:
            count+=int(1)
    accuracy = (len(validation_set)-count)/len(validation_set)
    print ('Validation set accuracy:%.4f'% (accuracy))
    f = open('maxEnt_classifier.pickle', 'wb')
    pickle.dump(classifier, f)
    f.close()
    print ('\nPredicting for test data')
    test_data = get_data_from_file(test_csv_file, isTrain=False)
    test_set_formatted = [(list_to_dict(element[0]), element[1]) for element in test_data]
    tweet_id = int(0)
    results = []
    for review in test_set_formatted:
        text = review[0]
        label = classifier.classify(text)
        results.append((str(tweet_id), label))
        tweet_id += int(1)
    save_results_to_csv(results, 'maxent.csv')
    print ('\nSaved to maxent.csv')

  ==> Training (1 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.565
         Final          -0.61597        0.755
  -1.010 andyhurleyday==True and label is '0'
   1.000 youtwitface==True and label is '0'
   1.000 alltimelowweek==True and label is '1'
   1.000 tired.its==True and label is '1'
   1.000 halfbloodprince==True and label is '1'
   1.000 hiccuuppss==True and label is '0'
   1.000 thaipbs==True and label is '1'
   1.000 harpersglobe==True and label is '1'
   1.000 twitterart==True and label is '1'
   1.000 ouchmyshoulder==True and label is '0'
Validation set accuracy:0.6971

Predicting for test data

Saved to maxent.csv


Decision Tree

In [18]:
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/decisiontree.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 6868/10000 = 68.6800 %


Random Forest

In [19]:
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/randomforest.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Testing
Processing 1/1
Correct: 7235/10000 = 72.3500 %


XGBoost

In [ ]:
#!pip2 install --upgrade xgboost

In [20]:
import random
import numpy as np
from xgboost import XGBClassifier
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer

# Performs classification using XGBoost.


FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
TRAIN = True
UNIGRAM_SIZE = 15000
VOCAB_SIZE = UNIGRAM_SIZE
USE_BIGRAMS = True
if USE_BIGRAMS:
    BIGRAM_SIZE = 10000
    VOCAB_SIZE = UNIGRAM_SIZE + BIGRAM_SIZE
FEAT_TYPE = 'frequency'


def get_feature_vector(tweet):
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in xrange(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


def extract_features(tweets, batch_size=500, test_file=True, feat_type='presence'):
    num_batches = int(np.ceil(len(tweets) / float(batch_size)))
    for i in xrange(num_batches):
        batch = tweets[i * batch_size: (i + 1) * batch_size]
        features = lil_matrix((batch_size, VOCAB_SIZE))
        labels = np.zeros(batch_size)
        for j, tweet in enumerate(batch):
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = bigrams.get(bigram)
                    if idx:
                        features[j, UNIGRAM_SIZE + idx] += 1
        yield features, labels


def apply_tf_idf(X):
    transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
    transformer.fit(X)
    return transformer


def process_tweets(csv_file, test_file=True):
    """Returns a list of tuples of type (tweet_id, feature_vector)
            or (tweet_id, sentiment, feature_vector)
    Args:
        csv_file (str): Name of processed csv file generated by preprocess.py
        test_file (bool, optional): If processing test file
    Returns:
        list: Of tuples
    """
    tweets = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, int(sentiment), feature_vector))
            write_status(i + 1, total)
    print ('\n')
    return tweets


if __name__ == '__main__':
    xrange =range
    np.random.seed(1337)
    unigrams = top_n_words(FREQ_DIST_FILE, UNIGRAM_SIZE)
    if USE_BIGRAMS:
        bigrams = top_n_bigrams(BI_FREQ_DIST_FILE, BIGRAM_SIZE)
    tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    if TRAIN:
        train_tweets, val_tweets = split_data(tweets)
    else:
        random.shuffle(tweets)
        train_tweets = tweets
    del tweets
    print ('Extracting features & training batches')
    clf = XGBClassifier(max_depth=25, silent=False, n_estimators=400)
    batch_size = len(train_tweets)
    i = 1
    n_train_batches = int(np.ceil(len(train_tweets) / float(batch_size)))
    for training_set_X, training_set_y in extract_features(train_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
        write_status(i, n_train_batches)
        i += 1
        if FEAT_TYPE == 'frequency':
            tfidf = apply_tf_idf(training_set_X)
            training_set_X = tfidf.transform(training_set_X)
        clf.fit(training_set_X, training_set_y)
    print ('\n')
    print ('Testing')
    if TRAIN:
        correct, total = 0, len(val_tweets)
        i = 1
        batch_size = len(val_tweets)
        n_val_batches = int(np.ceil(len(val_tweets) / float(batch_size)))
        for val_set_X, val_set_y in extract_features(val_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
            if FEAT_TYPE == 'frequency':
                val_set_X = tfidf.transform(val_set_X)
            prediction = clf.predict(val_set_X)
            correct += np.sum(prediction == val_set_y)
            write_status(i, n_val_batches)
            i += 1
        print ('\nCorrect: %d/%d = %.4f %%' % (correct, total, correct * 100. / total))
    else:
        del train_tweets
        test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)
        n_test_batches = int(np.ceil(len(test_tweets) / float(batch_size)))
        predictions = np.array([])
        print ('Predicting batches')
        i = 1
        for test_set_X, _ in extract_features(test_tweets, test_file=True, feat_type=FEAT_TYPE):
            if FEAT_TYPE == 'frequency':
                test_set_X = tfidf.transform(test_set_X)
            prediction = clf.predict(test_set_X)
            predictions = np.concatenate((predictions, prediction))
            write_status(i, n_test_batches)
            i += 1
        predictions = [(str(j), int(predictions[j]))
                       for j in range(len(test_tweets))]
        save_results_to_csv(predictions, 'xgboost.csv')
        print ('\nSaved to xgboost.csv')

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7688/10000 = 76.8800 %


SVM

In [21]:
!python2 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/svm.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7787/10000 = 77.8700 %


Multilayer Perceptron

In [22]:
from keras.models import Sequential, load_model
from keras.layers import Dense
import sys
import random
import numpy as np

# Performs classification using an MLP/1-hidden-layer NN.

FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
TRAIN = True
UNIGRAM_SIZE = 15000
VOCAB_SIZE = UNIGRAM_SIZE
USE_BIGRAMS = False
if USE_BIGRAMS:
    BIGRAM_SIZE = 10000
    VOCAB_SIZE = UNIGRAM_SIZE + BIGRAM_SIZE
FEAT_TYPE = 'frequency'


def get_feature_vector(tweet):
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in xrange(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


def extract_features(tweets, batch_size=500, test_file=True, feat_type='presence'):
    num_batches = int(np.ceil(len(tweets) / float(batch_size)))
    for i in xrange(num_batches):
        batch = tweets[i * batch_size: (i + 1) * batch_size]
        features = np.zeros((batch_size, VOCAB_SIZE))
        labels = np.zeros(batch_size)
        for j, tweet in enumerate(batch):
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = bigrams.get(bigram)
                    if idx:
                        features[j, UNIGRAM_SIZE + idx] += 1
        yield features, labels


def process_tweets(csv_file, test_file=True):
    tweets = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, int(sentiment), feature_vector))
            write_status(i + 1, total)
    print ('\n')
    return tweets


def build_model():
    model = Sequential()
    model.add(Dense(500, input_dim=VOCAB_SIZE, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    return model


def evaluate_model(model, val_tweets):
    correct, total = 0, len(val_tweets)
    for val_set_X, val_set_y in extract_features(val_tweets, feat_type=FEAT_TYPE, test_file=False):
        prediction = model.predict_on_batch(val_set_X)
        prediction = np.round(prediction)
        correct += np.sum(prediction == val_set_y[:, None])
    return float(correct) / total


if __name__ == '__main__':
    np.random.seed(1337)
    unigrams = top_n_words(FREQ_DIST_FILE, UNIGRAM_SIZE)
    if USE_BIGRAMS:
        bigrams = top_n_bigrams(BI_FREQ_DIST_FILE, BIGRAM_SIZE)
    tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    if TRAIN:
        train_tweets, val_tweets = split_data(tweets)
    else:
        random.shuffle(tweets)
        train_tweets = tweets
    del tweets
    print ('Extracting features & training batches')
    nb_epochs = 5
    batch_size = 500
    model = build_model()
    n_train_batches = int(np.ceil(len(train_tweets) / float(batch_size)))
    best_val_acc = 0.0
    for j in xrange(nb_epochs):
        i = 1
        for training_set_X, training_set_y in extract_features(train_tweets, feat_type=FEAT_TYPE, batch_size=batch_size, test_file=False):
            o = model.train_on_batch(training_set_X, training_set_y)
            sys.stdout.write('\rIteration %d/%d, loss:%.4f, acc:%.4f' %
                             (i, n_train_batches, o[0], o[1]))
            sys.stdout.flush()
            i += 1
        val_acc = evaluate_model(model, val_tweets)
        print ('\nEpoch: %d, val_acc:%.4f' % (j + 1, val_acc))
        random.shuffle(train_tweets)
        if val_acc > best_val_acc:
            print ('Accuracy improved from %.4f to %.4f, saving model' % (best_val_acc, val_acc))
            best_val_acc = val_acc
            model.save('best_model.h5')
    print ('Testing')
    del train_tweets
    del model
    model = load_model('best_model.h5')
    test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)
    n_test_batches = int(np.ceil(len(test_tweets) / float(batch_size)))
    predictions = np.array([])
    print ('Predicting batches')
    i = 1
    for test_set_X, _ in extract_features(test_tweets, feat_type=FEAT_TYPE, batch_size=batch_size, test_file=True):
        prediction = np.round(model.predict_on_batch(test_set_X).flatten())
        predictions = np.concatenate((predictions, prediction))
        write_status(i, n_test_batches)
        i += 1
    predictions = [(str(j), int(predictions[j]))
                   for j in range(len(test_tweets))]
    save_results_to_csv(predictions, '1layerneuralnet.csv')
    print ('\nSaved to 1layerneuralnet.csv')

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Iteration 180/180, loss:0.4980, acc:0.7580
Epoch: 1, val_acc:0.7575
Accuracy improved from 0.0000 to 0.7575, saving model
Iteration 180/180, loss:0.5031, acc:0.7880
Epoch: 2, val_acc:0.7660
Accuracy improved from 0.7575 to 0.7660, saving model
Iteration 180/180, loss:0.4444, acc:0.8020
Epoch: 3, val_acc:0.7688
Accuracy improved from 0.7660 to 0.7688, saving model
Iteration 180/180, loss:0.4643, acc:0.7780
Epoch: 4, val_acc:0.7643
Iteration 180/180, loss:0.4644, acc:0.7820
Epoch: 5, val_acc:0.7685
Testing
Generating feature vectors
Processing 300000/300000

Predicting batches
Processing 600/600
Saved to 1layerneuralnet.csv


**Reccurent Neural Networks**

In [12]:
!python3 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/lstm.py

Looking for GLOVE vectors
Processing 1193515/0

Found 38248 words in GLOVE
Generating feature vectors
Processing 100000/100000

2022-01-28 05:07:19.571135: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 25)            2250025   
                                                                 
 dropout (Dropout)           (None, 40, 25)            0         
                                                                 
 lstm (LSTM)                 (None, 128)               78848     
                                                                 
 dense (Dense)               (None, 64)                8256      
                                        

**CNN**

In [13]:
!python3 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/cnn.py

Looking for GLOVE seeds
Processing 1193515/0

Generating feature vectors
Processing 100000/100000

2022-01-28 05:09:27.186738: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Epoch 1/3
704/704 [==============================] - ETA: 0s - loss: 0.5844 - accuracy: 0.6790
Epoch 00001: loss improved from inf to 0.58443, saving model to /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/models/cnn.h5
704/704 [==============================] - 28s 35ms/step - loss: 0.5844 - accuracy: 0.6790 - val_loss: 0.5309 - val_accuracy: 0.7192 - lr: 0.0010
Epoch 2/3
702/704 [============================>.] - ETA: 0s - loss: 0.5186 - accuracy: 0.7415
Epoch 00002: loss improved from 0.58443 to 0.51862, saving model to /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/models/cnn.h5
704/704 [==============================] - 24s 34ms/s

Majority Vote Ensemble

In [15]:
!python3 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/extract-cnn-feats.py /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/models/cnn.h5

Looking for GLOVE seeds
Processing 1193515/0

Generating feature vectors
Processing 100000/100000

2022-01-28 05:20:48.226536: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_input (InputLayer  [(None, 40)]             0         
 )                                                               
                                                                 
 embedding (Embedding)       (None, 40, 25)            2250025   
                                                                 
 dropout (Dropout)           (None, 40, 25)            0         
                                                                 
 conv1d (Conv1D)             (None, 38, 600)           45600     
        

In [16]:
!python3 /content/drive/MyDrive/UAS_ML/Twitter-Sentiment-Analysis-main/code/cnn-feats-svm.py

(90000, 600) (90000,) (10000, 600) (10000,)
....................................................................................................
optimization finished, #iter = 1000

Using -s 2 may be faster (also see FAQ)

Objective value = -50646.502941
nSV = 78618
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
[LibLinear]LinearSVC(C=1, verbose=1)
Val Accuracy: 79.92%
